In [ ]:
# Import Required Packages
%matplotlib inline
import numpy as np
#import networkx as nx
import time
import random
import json
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from sympy import *
from tqdm.notebook import tqdm

#From Qiskit library
from qiskit import *
from qiskit.visualization import *
from qiskit import IBMQ
from qiskit import BasicAer
from qiskit import Aer, execute
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.quantum_info import Statevector
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.circuit import Parameter
from qiskit.optimization.converters import IsingToQuadraticProgram
from qiskit.aqua.components.optimizers import NELDER_MEAD, CRS, ADAM

In [ ]:
qiskit.qiskit.__qiskit_version__

In [ ]:
#Function: Load JSON file
def loadjson( filename ):
    loaded_file = open(filename, "r")
    json_data = json.load(loaded_file)
    loaded_file.close()
    return json_data

#Function: Load Pickle file
def loadpkl( filename ):
    loaded_file = open(filename, "rb")
    pkl_data = pickle.load( loaded_file )
    loaded_file.close()
    return pkl_data

#Function: Get single term coefficients (to 4 decimal places) from Sympy Expression 
def get_coeff( variable, expression ):
    poly = Poly(expression.coeff(variable), domain = 'RR')
    return round(float(poly.coeffs()[len(poly.coeffs())-1]), 4)

#Function: Get coefficient of quadratic term (to 4 decimal places) from Sympy Expression
def quad_coeff( var_1, var_2, expression ):
    return round(float(expression.coeff(var_1).coeff(var_2)), 4)

In [ ]:
pwd

In [ ]:
import sys
sys.executable

In [ ]:
#Read and store files into variables
car_routes = loadjson( "3cars3routes.json" )
# H = loadjson( "Hamiltonian9qbits.json")
[objective, Ham, objective_normed, Ham_normed] = loadpkl("expressions.pkl")
[total_cost, constraints] = loadpkl("cost_constraints.pkl")

In [ ]:
Q = IndexedBase('Q')
car_routes_variables = {}
for route in car_routes:
    car_routes_variables[route] = Q[route[3],route[10]]
car_routes_variables

In [ ]:
no_routes = 3
no_cars = 3
no_qubits = no_routes * no_cars

In [ ]:
#Associate binary variable to each Q11, Q12, etc with one index for QuadraticProgram class
binary_vars = {}
for variable in car_routes_variables:
    binary_vars[car_routes_variables[variable]] = 'X' + str(no_routes*(int(variable[3])-1)+int(variable[10]))
Qvariables = tuple(binary_vars.keys())
Xvariables = tuple(binary_vars.values())

In [ ]:
#Construct Quadratic Problem using known (quadratic) objective function
qubo1 = QuadraticProgram()
linear = []
for variable in car_routes_variables.values():
    qubo1.binary_var( binary_vars[variable] )
    linear.append( get_coeff(variable, objective) )
quadratic = {}
for i in range(no_qubits):
    for j in range(i+1, no_qubits):
        pairing = ( Xvariables[i], Xvariables[j] )
        coeff = quad_coeff( Qvariables[i], Qvariables[j], objective )
        if int(coeff) != 0:
            quadratic[pairing] = coeff
qubo1.minimize(linear = linear, quadratic = quadratic)

In [ ]:
op, offset = qubo1.to_ising()

In [ ]:
coeffs = [abs(op.oplist[i].coeff) for i in range(len(op.oplist))]
norm_factor = np.amax(coeffs)

In [ ]:
#Normalised objective function
qubo = QuadraticProgram()
linear = []
for variable in car_routes_variables.values():
    qubo.binary_var( binary_vars[variable] )
    linear.append( (get_coeff(variable, objective)/norm_factor) )
quadratic = {}
for i in range(no_qubits):
    for j in range(i+1, no_qubits):
        pairing = ( Xvariables[i], Xvariables[j] )
        coeff = quad_coeff( Qvariables[i], Qvariables[j], objective )
        if int(coeff) != 0:
            quadratic[pairing] = (coeff/norm_factor)
qubo.minimize(linear = linear, quadratic = quadratic)

In [ ]:
op, offset = qubo.to_ising()

In [ ]:
print(op)

In [ ]:
qaoa_mes = QAOA(quantum_instance=Aer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [ ]:
%%time
exact_result = exact.solve(qubo)
print(exact_result)

In [ ]:
def eval_cost(x, linear, quadratic):
    x1 = int(x[0])
    x2 = int(x[1])
    x3 = int(x[2])
    x4 = int(x[3])
    x5 = int(x[4])
    x6 = int(x[5])
    x7 = int(x[6])
    x8 = int(x[7])
    x9 = int(x[8])
    cost = 0
    for i in range(len(linear)):
        cost += int(x[no_qubits-i-1])*linear[i]
    for term in quadratic:
        i = int(term[0][1])-1
        j = int(term[1][1])-1
        cost += int(x[no_qubits-i-1])*int(x[no_qubits-j-1])*quadratic[term]
    return cost

In [ ]:
#Print first 20 lowest states
values = {}
for k in range(2**9):
    string = '{0:09b}'.format(k)
    values[string] = eval_cost(string, linear, quadratic)
sort_values = sorted(values.items(), key=lambda x: x[1])
i=0
for string, cost in sort_values:
    print(string, round(cost,3), string[0:3].count('1')==1 and string[3:6].count('1')==1 and string[6:9].count('1')==1)
    i += 1
    if i == 20:
        break

In [ ]:
#Use statevector with GPU to simulate slightly faster for less than ~25 qubits
#If more than 25 qubits, use method = "statevector" (CPU) instead.

simulator = QasmSimulator()
optimizer = NELDER_MEAD(maxiter = 2)

In [ ]:
#Construct QAOA circuit of size p given operator from qubo problem ( operator is obtained from QuadraticProblem.to_ising() )
def construct_QAOAcirc( p, operator ):
    #Initialise parameter list
    params_expr = []
    
    #Create arbitrary "gamma_n" parameters
    for a in range(p):
        params_expr.append( Parameter('a'+str(a+1)) )
    #Likewise for "beta_n" parameters
    for b in range(p):
        params_expr.append( Parameter('b'+str(b+1)) )
        
    #Create circuit given operator, gamma, and beta parameters
    qaoa = QAOA(operator = operator, p=p, quantum_instance=Aer.get_backend('qasm_simulator'))
    circuit = qaoa.construct_circuit( params_expr[0:2*p] )[0]
    circuit.measure_active()
    return p, params_expr, circuit


#Find expectation value, given qaoa_circuit, qubo_problem and freq_params (u1, u2, ... , v1, v2, ...)
def expectation( qaoa_circuit, qubo_problem, freq_params ):
    
    #unload p, params_expr, circuit from the return value of contruct_QAOAcirc
    p = int(qaoa_circuit[0])
    params_expr = qaoa_circuit[1]
    circuit = qaoa_circuit[2]
    params = convert_fourier_parameters(p,p,freq_params)
    if len(params) != 2*p:
        print("Must have len(params) = 2*no_layers")
    else:    
        circuit2 = circuit.assign_parameters({params_expr[i]: params[i] for i in range(len(params))}, inplace = False)
        no_shots = 100000
        result = execute(circuit2, simulator, shots = no_shots).result()
        counts = result.get_counts()
        exp_val = []
        for state in counts:
            a = counts[state]
            b = eval_cost(state, linear, quadratic)
            exp_val.append(a*b)        
        exp_val = np.sum(exp_val)/no_shots
        print(".", end = "")
        return np.round(exp_val, decimals = 1)

#Convert frequency parameters to standard gamma and beta parameters
def convert_fourier_parameters(p , q, frequency_parameters):
    u_params = np.array(frequency_parameters[0:q], dtype = float)
    v_params = np.array(frequency_parameters[q:2*q], dtype = float)
    gamma_params = np.empty((p,), dtype = np.float32)
    beta_params = np.empty((p,), dtype = np.float32)
    for i in range(p):
        gamma_inter, beta_inter = 0,0
        a = (i+0.5)*np.pi/p
        for k in range(q):
            gamma_inter += u_params[k]*np.sin( (k+0.5)*a )
            beta_inter += v_params[k]*np.cos( (k+0.5)*a )
        gamma_params[i] = gamma_inter
        beta_params[i] = beta_inter
    return np.append(gamma_params, beta_params)

Inputs: QAOA problem, no_layers, u_v_L, u_v_B, no_perturb

1. Unload operator from QAOA problem
2. Construct circuit with parameters for no_layers
3. Generate no_perturb number of parameters about u_v_B
4. Optimize around the u_v_L, u_v_B, and the perturbed u_v_B's
5. Choose best for the new u_v_B, and also the new u_v_L
6. Return the new u_v_B, and the new u_v_B

Note: Make sure u_v_L = (u1, u2, ..., v1, v2, ...) has 2*(no_layers - 1) 

In [ ]:
def optimize_qubo(qubo_problem, p, freq_optimal_params_L, freq_optimal_params_B, no_perturb):
    perturb_scale_factor = 0.6
    op, offset = qubo_problem.to_ising()
    generator = np.random.default_rng()
    variable_bounds = [ (-np.pi/2, np.pi/2) for _ in range(2*p)]
    if len(freq_optimal_params_L) != len(freq_optimal_params_B) and len(freq_optimal_params_B) != (p-1)*2:
        print('Check number of input parameters are correct')
        return None
    
    #Construct circuit
    print(f'Step {p}: Building circuit...')
    circuit = construct_QAOAcirc(p, op)
    
    #New initial points generated from previous initial points
    params = np.zeros(shape = (2*p,), dtype=float)+0.05  
    initial_freq_params_L = params.copy()
    initial_freq_params_B_0 = params.copy()
    initial_freq_params_L[0:p-1] = freq_optimal_params_L[0:p-1]
    initial_freq_params_L[p:2*p-1] = freq_optimal_params_L[p-1:2*p-2]
    initial_freq_params_B_0[0:p-1] = freq_optimal_params_B[0:p-1]
    initial_freq_params_B_0[p:2*p-1] = freq_optimal_params_B[p-1:2*p-2]
    
    #Generate perturbed initial points     
    if p != 1:
        initial_freq_params_B = np.empty(shape = (no_perturb+2, ), dtype = object)
        initial_freq_params_B[0] = initial_freq_params_B_0
        u_random = np.array([np.array(generator.normal(0, freq_optimal_params_B[i]**2, no_perturb+1)) for i in range(0,p-1)])
        v_random = np.array([np.array(generator.normal(0, freq_optimal_params_B[i]**2, no_perturb+1)) for i in range(p-1,2*p-2)])
        m = len(initial_freq_params_B)
        for j in range(1,m-1):
            perturbed_params = params.copy()
            perturbed_params[0:p-1] = initial_freq_params_B_0[0:p-1] + perturb_scale_factor*u_random[:,j]
            perturbed_params[p:2*p-1] = initial_freq_params_B_0[p:2*p-1] + perturb_scale_factor*v_random[:,j]
            initial_freq_params_B[j] = perturbed_params
        initial_freq_params_B[m-1] = initial_freq_params_B[0]
        
    print(f'Step {p}: Now optimising parameters (Nelder-Mead)...')
    ##Define obj_function that inputs only parameters
    obj_expectation = lambda x: expectation( circuit, qubo_problem, x ) 
    print('\n'+'initial freq point L:', initial_freq_params_L)
    optimal_result = optimizer.optimize(num_vars = 2*p,
                                        objective_function = obj_expectation,
                                        variable_bounds = variable_bounds, 
                                        initial_point = initial_freq_params_L)
    optimal_params_L, exp_val_L, no_evals = optimal_result
    
    if p == 1:
        optimal_params_B, exp_val_B = optimal_params_L, exp_val_L
    
    else:
        perturbed_params = [optimal_params_L]
        perturbed_exp_vals = [exp_val_L]
        print(initial_freq_params_B)
        for l in range(m):
            print('\n'+f'{l}, initial freq point B: ', initial_freq_params_B[l])
            if l == m-1:
                break
            perturbed_optimal_result = optimizer.optimize(num_vars = 2*p,
                                    objective_function = obj_expectation,
                                    variable_bounds= variable_bounds, 
                                    initial_point=initial_freq_params_B[l])
            print('\nNelderMeadcomplete')
            perturbed_optimal_params, perturbed_optimal_expval = perturbed_optimal_result[0:2] 
            perturbed_params.append(perturbed_optimal_params)
            perturbed_exp_vals.append(perturbed_exp_vals)
        exp_val_B = np.amin(perturbed_exp_vals)
        minim_index = np.awhere(perturbed_exp_vals == exp_val_B)[0][0]
        optimal_params_B = perturbed_params[minim_index]
    
    return optimal_params_L, optimal_params_B

In [ ]:
optimal_params_L_1, optimal_params_B_1 = optimize_qubo(qubo, 1, [], [], 1)

In [ ]:
optimal_params_L_2, optimal_params_B_2 = optimize_qubo(qubo, 2, optimal_params_L_1, optimal_params_B_1, 1)

In [ ]:
print(optimal_params_L_1)
print(optimal_params_B_1)

In [ ]:
a = construct_QAOAcirc(1, op)

In [ ]:
a[0]

In [ ]:
simulator = QasmSimulator(method = "statevector_gpu")

In [ ]:
%%time
expectation(a, qubo, np.array([0.05,0.05]))